# Single cell TCR/RNA-seq data analysis using Scirpy

* __Notebook version__: `v0.0.1`
* __Created by:__ `Imperial BRC Genomics Facility`
* __Maintained by:__ `Imperial BRC Genomics Facility`
* __Docker image:__ `imperialgenomicsfacility/scirpy-notebook-image:release-v0.0.1`
* __Github repository:__ [imperial-genomics-facility/scirpy-notebook-image](https://github.com/imperial-genomics-facility/scirpy-notebook-image)
* __Created on:__ {{ DATE_TAG }}
* __Contact us:__ [Imperial BRC Genomics Facility](https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/contact/)
* __License:__ [Apache License 2.0](https://github.com/imperial-genomics-facility/scirpy-notebook-image/blob/master/LICENSE)
* __Project name:__ {{ PROJECT_IGF_ID }}
{% if SAMPLE_IGF_ID %}* __Sample name:__ {{ SAMPLE_IGF_ID }}{% endif %}

## Table of contents

* [Introduction](#Introduction)
* [Tools required](#Tools-required)
* [Input parameters](#Input-parameters)
* [Loading required libraries](#Loading-required-libraries)
* [Reading data from Cellranger output](#Reading-data-from-Cellranger-output)
* [Data processing and QC](#Data-processing-and-QC)
  * [Checking highly variable genes](#Checking-highly-variable-genes)
  * [Computing metrics for cell QC](#Computing-metrics-for-cell-QC)
  * [Ploting MT gene fractions](#Ploting-MT-gene-fractions)
  * [Counting cells per gene](#Counting-cells-per-gene)
  * [Plotting count depth vs MT fraction](#Plotting-count-depth-vs-MT-fraction)
  * [Checking thresholds and filtering data](#Checking-thresholds-and-filtering-data)
* [Gene expression data analysis](#Gene-expression-data-analysis)
  * [Normalization](#Normalization)
  * [Highly variable genes](#Highly-variable-genes)
  * [Regressing out technical effects](#Regressing-out-technical-effects)
  * [Principal component analysis](#Principal-component-analysis)
  * [Neighborhood graph](#Neighborhood-graph)
    * [Clustering the neighborhood graph](#Clustering-the-neighborhood-graph)
    * [Embed the neighborhood graph using 2D UMAP](#Embed-the-neighborhood-graph-using-2D-UMAP)
  * [Finding marker genes](#Finding-marker-genes)
  * [Cell annotation](#Cell-annotation)
* [VDJ data analysis](#VDJ-data-analysis)
  * [TCR quality control](#TCR-quality-control)
  * [Compute CDR3 neighborhood graph and define clonotypes](#Compute-CDR3-neighborhood-graph-and-define-clonotypes)
  * [Re-compute CDR3 neighborhood graph and define clonotype clusters](#Re-compute-CDR3-neighborhood-graph-and-define-clonotype-clusters)
  * [Including the V-gene in clonotype definition](#Including-the-V-gene-in-clonotype-definition)
  * [Clonotype analysis](#Clonotype-analysis)
    * [Clonal expansion](#Clonal-expansion)
    * [Clonotype abundance](#Clonotype-abundance)
  * [Gene usage](#Gene-usage)
  * [Spectratype plots](#Spectratype-plots)


## Introduction
This notebook for running single cell TCR/RNA-seq data analysis (for a single sample) using Scirpy and Scanpy package. Most of the codes and documentation used in this notebook has been copied from the following sources:

* [Analysis of 3k T cells from cancer](https://icbi-lab.github.io/scirpy/tutorials/tutorial_3k_tcr.html)
* [Scanpy - Preprocessing and clustering 3k PBMCs](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html)
* [Single-cell-tutorial](https://github.com/theislab/single-cell-tutorial)

## Tools required
* [Scirpy](https://icbi-lab.github.io/scirpy/index.html)
* [Scanpy](https://scanpy-tutorials.readthedocs.io/en/latest)
* [Plotly](https://plotly.com/python/)
* [UCSC Cell Browser](https://pypi.org/project/cellbrowser/)

## Input parameters

In [ ]:
cell_ranger_count_path = '{{ CELLRANGER_COUNT_DIR }}'
cell_ranger_vdj_path = '{{ CELLRANGER_VDJ_DIR }}'
cell_marker_list = '{{ CELL_MARKER_LIST }}'
genome_build = '{{ GENOME_BUILD }}'

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

## Loading required libraries

We need  to load all the required libraries to environment before we can run any of the analysis steps. Also, we are checking the version information for most of the major packages used for analysis.

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import scanpy as sc
import scirpy as ir
import seaborn as sns
from matplotlib import pyplot as plt
sns.set_theme(context='notebook', style='darkgrid', palette='colorblind')

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

## Reading data from Cellranger output

Load the Cellranger VDJ output to Scanpy

In [ ]:
adata_tcr = \
  ir.io.read_10x_vdj(os.path.join(cell_ranger_vdj_path,"filtered_contig_annotations.csv"))

Load the Cellranger output to Scanpy

In [ ]:
adata = \
  sc.read_10x_h5(os.path.join(cell_ranger_count_path,"filtered_feature_bc_matrix.h5"))

Converting the gene names to unique values

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata_tcr.shape

Checking the data dimensions before checking QC

In [ ]:
adata.shape

In [ ]:
ir.pp.merge_with_ir(adata, adata_tcr)

Scanpy stores the count data is an annotated data matrix ($observations$ e.g. cell barcodes × $variables$ e.g gene names) called [AnnData](https://anndata.readthedocs.io) together with annotations of observations($obs$), variables ($var$) and unstructured annotations ($uns$)

In [ ]:
adata.obs.head()

In [ ]:
adata.var.head()

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

## Data processing and QC

### Checking highly variable genes

Computing fraction of counts assigned to each gene over all cells. The top genes with the highest mean fraction over all cells are
plotted as boxplots.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,8))
sc.pl.highest_expr_genes(adata, n_top=20,ax=ax)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Computing metrics for cell QC

Listing the Mitochondrial genes detected in the cell population

In [ ]:
mt_genes = 0
mt_genes = [gene for gene in adata.var_names if gene.startswith('MT-')]
mito_genes = adata.var_names.str.startswith('MT-')
if len(mt_genes)==0:
    print('Looking for mito genes with "mt-" prefix')
    mt_genes = [gene for gene in adata.var_names if gene.startswith('mt-')]
    mito_genes = adata.var_names.str.startswith('mt-')

if len(mt_genes)==0:
    print("No mitochondrial genes found")
else:
    print("Mitochondrial genes: count: {0}, lists: {1}".format(len(mt_genes),mt_genes))

Typical quality measures for assessing the quality of a cell includes the following components
* Number of molecule counts (UMIs or $n\_counts$ )
* Number of expressed genes ($n\_genes$)
* Fraction of counts that are mitochondrial ($percent\_mito$)

We are calculating the above mentioned details using the following codes

In [ ]:
adata.obs['mito_counts'] =  np.sum(adata[:, mito_genes].X, axis=1).A1
adata.obs['percent_mito'] = \
  np.sum(adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
adata.obs['n_counts'] = adata.X.sum(axis=1).A1
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
adata.obs['n_genes'] = (adata.X > 0).sum(1)

Checking $obs$ section of the AnnData object again

In [ ]:
adata.obs.head()

Sorting barcodes based on the $percent\_mito$ column

In [ ]:
adata.obs.sort_values('percent_mito',ascending=False).head()

A high fraction of mitochondrial reads being picked up can indicate cell stress, as there is a low proportion of nuclear mRNA in the cell. It should be noted that high mitochondrial RNA fractions can also be biological signals indicating elevated respiration. <p/>

Cell barcodes with high count depth, few detected genes and high fraction of mitochondrial counts may indicate cells whose cytoplasmic mRNA has leaked out due to a broaken membrane and only the mRNA located in the mitochrondia has survived. <p/>

Cells with high UMI counts and detected genes may represent dublets (it requires further checking).

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Ploting MT gene fractions

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
sc.pl.violin(\
  adata,
  ['n_genes', 'n_counts', 'percent_mito'],
  jitter=0.4,
  log=True,
  stripplot=True,
  show=False,
  multi_panel=False)

Violin plot (above) shows the computed quality measures of UMI counts, gene counts and fraction of mitochondrial counts.

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
ax = sc.pl.scatter(adata, 'n_counts', 'n_genes', color='percent_mito',show=False)
ax.set_title('Fraction mitochondrial counts', fontsize=12)
ax.set_xlabel("Count depth",fontsize=12)
ax.set_ylabel("Number of genes",fontsize=12)
ax.tick_params(labelsize=12)
ax.axhline(500, 0,1, color='red')
ax.axvline(1000, 0,1, color='red')

The above scatter plot shows number of genes vs number of counts with $MT$ fraction information. We will be using a cutoff of 1000 counts and 500 genes (<span style="color:red">red lines</span>) to filter out dying cells. 

In [ ]:
ax = sc.pl.scatter(adata[adata.obs['n_counts']<10000], 'n_counts', 'n_genes', color='percent_mito',show=False)
ax.set_title('Fraction mitochondrial counts', fontsize=12)
ax.set_xlabel("Count depth",fontsize=12)
ax.set_ylabel("Number of genes",fontsize=12)
ax.tick_params(labelsize=12)
ax.axhline(500, 0,1, color='red')
ax.axvline(1000, 0,1, color='red')

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Counting cells per gene

In [ ]:
adata.var['cells_per_gene'] = np.sum(adata.X > 0, 0).T

ax = sns.histplot(adata.var['cells_per_gene'][adata.var['cells_per_gene'] < 100], kde=False, bins=60)
ax.set_xlabel("Number of cells",fontsize=12)
ax.set_ylabel("Frequency",fontsize=12)
ax.set_title('Cells per gene', fontsize=12)
ax.tick_params(labelsize=12)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Plotting count depth vs MT fraction

The scatter plot showing the count depth vs MT fraction counts and the <span style="color:red">red line</span> shows the default cutoff value for MT fraction 0.2

In [ ]:
ax = sc.pl.scatter(adata, x='n_counts', y='percent_mito',show=False)
ax.set_title('Count depth vs Fraction mitochondrial counts', fontsize=12)
ax.set_xlabel("Count depth",fontsize=12)
ax.set_ylabel("Fraction mitochondrial counts",fontsize=12)
ax.tick_params(labelsize=12)
ax.axhline(0.2, 0,1, color='red')

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Checking thresholds and filtering data

Now we need to filter the cells which doesn't match our thresholds.

In [ ]:
min_counts_threshold = 1000
max_counts_threshold = 50000
min_gene_counts_threshold = 500
max_mito_pct_threshold = 0.2

if adata[adata.obs['n_counts'] > min_counts_threshold].n_obs < 1000:
    min_counts_threshold = 100

if adata[adata.obs['n_counts'] < max_counts_threshold].n_obs < 1000:
    max_counts_threshold = 80000
    
if adata[adata.obs['n_genes'] > min_gene_counts_threshold].n_obs < 1000:
    min_gene_counts_threshold = 100
    
if adata[adata.obs['percent_mito'] < max_mito_pct_threshold].n_obs < 1000:
    max_mito_pct_threshold = 0.5


In [ ]:
if min_counts_threshold != 1000 or min_gene_counts_threshold!= 500:
  print("Resetting the thresholds")
  ax = sc.pl.scatter(adata, 'n_counts', 'n_genes', color='percent_mito',show=False)
  ax.set_title('Fraction mitochondrial counts', fontsize=12)
  ax.set_xlabel("Count depth",fontsize=12)
  ax.set_ylabel("Number of genes",fontsize=12)
  ax.tick_params(labelsize=12)
  ax.axhline(min_gene_counts_threshold, 0,1, color='red')
  ax.axvline(min_counts_threshold, 0,1, color='red')

In [ ]:
print('Total number of cells: {0}'.format(adata.n_obs))
print('Filtering dataset using thresholds')

sc.pp.filter_cells(adata, min_counts = min_counts_threshold)
print('Number of cells after min count ({0}) filter: {1}'.format(min_counts_threshold,adata.n_obs))

sc.pp.filter_cells(adata, max_counts = max_counts_threshold)
print('Number of cells after max count ({0}) filter: {1}'.format(max_counts_threshold,adata.n_obs))

sc.pp.filter_cells(adata, min_genes = min_gene_counts_threshold)
print('Number of cells after gene ({0}) filter: {1}'.format(min_gene_counts_threshold,adata.n_obs))

adata = adata[adata.obs['percent_mito'] < max_mito_pct_threshold]
print('Number of cells after MT fraction ({0}) filter: {1}'.format(max_mito_pct_threshold,adata.n_obs))

print('Total number of cells after filtering: {0}'.format(adata.n_obs))

Also, we need to filter out any genes that are detected in only less than 20 cells. This operation reduces the dimensions of the matrix by removing zero count genes which are not really informative.

In [ ]:
min_cell_per_gene_threshold = 20

print('Total number of genes: {0}'.format(adata.n_vars))

sc.pp.filter_genes(adata, min_cells=min_cell_per_gene_threshold)
print('Number of genes after cell filter: {0}'.format(adata.n_vars))

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

## Gene expression data analysis

### Normalization

We are using a simple total-count based normalization (library-size correct) to transform the data matrix $X$ to 10,000 reads per cell, so that counts become comparable among cells.

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

Then logarithmize the data matrix

In [ ]:
sc.pp.log1p(adata)

Copying the normalized and logarithmized raw gene expression data to the .raw attribute of the AnnData object for later use.

In [ ]:
adata.raw = adata

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Highly variable genes

Following codes blocks are used to identify the highly variable genes (HGV) to further reduce the dimensionality of the dataset and to include only the most informative genes. HGVs will be used for clustering, trajectory inference, and dimensionality reduction/visualization.

We use a 'seurat' flavor based HGV detection step. Then, we run the following codes to do the actual filtering of data. The plots show how the data was normalized to select highly variable genes irrespective of the mean expression of the genes. This is achieved by using the index of dispersion which divides by mean expression, and subsequently binning the data by mean expression and selecting the most variable genes within each bin.

In [ ]:
plt.rcParams['figure.figsize']=(7,7)
sc.pp.highly_variable_genes(adata, flavor='seurat', min_mean=0.0125, max_mean=3, min_disp=0.5)
seurat_hgv = np.sum(adata.var['highly_variable'])
print("Counts of HGVs: {0}".format(seurat_hgv))
sc.pl.highly_variable_genes(adata)

In [ ]:
adata = adata[:, adata.var.highly_variable]

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Regressing out technical effects

Normalization scales count data to make gene counts comparable between cells. But it still contain unwanted variability. One of the most prominent technical covariates in single-cell data is count depth. Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed can improve the performance of trajectory inference algorithms.

In [ ]:
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])

Scale each gene to unit variance. Clip values exceeding standard deviation 10.

In [ ]:
sc.pp.scale(adata, max_value=10)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Principal component analysis

Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
plt.rcParams['figure.figsize']=(10,8)
sc.pl.pca(adata,color=[adata.var_names[0]])

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells.

The first principle component captures variation in count depth between cells and its marginally informative.

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix.

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Neighborhood graph
Computing the neighborhood graph of cells using the PCA representation of the data matrix.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

#### Clustering the neighborhood graph

Scanpy documentation recommends the Leiden graph-clustering method (community detection based on optimizing modularity) by Traag *et al.* (2018). Note that Leiden clustering (using `resolution=0.5`) directly clusters the neighborhood graph of cells, which we have already computed in the previous section.

In [ ]:
sc.tl.leiden(adata,resolution=0.5)

In [ ]:
cluster_length = len(adata.obs['leiden'].value_counts().to_dict().keys())
cluster_colors = sns.color_palette('colorblind',cluster_length,as_cmap=True)

### Embed the neighborhood graph using 2D UMAP

Scanpy documentation suggests embedding the graph in 2 dimensions using UMAP (McInnes et al., 2018), see below. It is potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preservers trajectories.

In [ ]:
sc.tl.umap(adata,n_components=2)

In [ ]:
gene_name = ['CST3']
if 'CST3' not in adata.var_names:
  gene_name = [adata.var_names[0]]

plt.rcParams['figure.figsize']=(10,8)
sc.pl.umap(adata,color=gene_name,size=100)

You can replace the `['CST3']` in the previous cell with your preferred list of genes.

e.g. `['LTB','IL32','CD3D']`

or may be with a Python onliner code to extract gene names with a specific prefix

e.g.

```python
sc.pl.umap(adata, color=[gene for gene in adata.var_names if gene.startswith('CD')], ncols=2)
```

Plot the scaled and corrected gene expression by `use_raw=False` and color them based on the leiden clusters.

In [ ]:
sc.pl.umap(adata,color='leiden',use_raw=False,palette=cluster_colors,size=100)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Finding marker genes

Let us compute a ranking for the highly differential genes in each cluster. For this, by default, the `.raw` attribute of AnnData is used in case it has been initialized before. We are using ` Wilcoxon rank-sum test` (Sonison & Robinson (2018) here.

In [ ]:
plt.rcParams['figure.figsize']=(6,6)
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, ncols=2)

Show the 10 top ranked genes per cluster 0, 1, …, N in a dataframe

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Cell annotation

Downloaded the cell type gene expression markers form [PanglaoDB](https://panglaodb.se/markers.html?cell_type=%27all_cells%27) and generated cell annotation plot

In [ ]:
df = pd.read_csv(cell_marker_list,delimiter='\t')

In [ ]:
if genome_build.upper() == 'HG38':
  print('Selecting human specific genes')
  markers_df = df[(df['species'] == 'Hs') | (df['species'] == 'Mm Hs')]
elif genome_build.upper() =='MM10':
  print('Selecting mouse specific genes')
  markers_df = df[(df['species'] == 'Mm') | (df['species'] == 'Mm Hs')]
else:
  raise ValueError('Species {0} not supported'.format(genome_build))
markers_df.shape

Collect the unique cell type list

In [ ]:
cell_types = list(markers_df['cell type'].unique())

Generate cell annotation using the marker gene list

In [ ]:
markers_dict = {}
for ctype in cell_types:
    df = markers_df[markers_df['cell type'] == ctype]
    markers_dict[ctype] = df['official gene symbol'].to_list()
cell_annotation = sc.tl.marker_gene_overlap(adata, markers_dict, key='rank_genes_groups',top_n_markers=20)
cell_annotation_norm = sc.tl.marker_gene_overlap(adata, markers_dict, key='rank_genes_groups', normalize='reference',top_n_markers=20)

In [ ]:
plt.rcParams['figure.figsize']=(10,10)
sns.heatmap(cell_annotation_norm.loc[cell_annotation_norm.sum(axis=1) > 0.05,], cbar=False, annot=True)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

## VDJ data analysis

### TCR quality control

While most of T cell receptors have exactly one pair of α and β chains, up to one third of T cells can have dual TCRs, i.e. two pairs of receptors originating from different alleles. 

Using the `scirpy.tl.chain_qc()` function, we can add a summary about the T cell receptor compositions to `adata.obs`.

__Chain pairing:__
* Orphan chain refers to cells that have either a single alpha or beta receptor chain.
* Extra chain refers to cells that have a full alpha/beta receptor pair, and an additional chain.
* Multichain refers to cells with more than two receptor pairs detected. These cells are likely doublets.


In [ ]:
ir.tl.chain_qc(adata)

Following plot shows the receptor types

In [ ]:
ir.pl.group_abundance(adata, groupby="receptor_type", target_col="leiden")

Use `groupby="receptor_subtype"` to check receptor subtypes, e.g. if the dataset contains only α/β T-cell receptor

In [ ]:
ir.pl.group_abundance(adata, groupby="receptor_subtype", target_col="leiden")

Checking for chain pairing

In [ ]:
ir.pl.group_abundance(adata, groupby="chain_pairing", target_col="leiden")

Check the fraction of productive T-cell receptors:

In [ ]:
print(
    "Fraction of cells with more than one pair of TCRs: {0}".format(
        np.sum(
            adata.obs["chain_pairing"].isin(
                ["extra VJ", "extra VDJ", "two full chains"]
            )
        )
        / adata.n_obs
    )
)

Visualize the Multichain cells on the UMAP plot and exclude them from downstream analysis:

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
sc.pl.umap(adata, color="multi_chain")

In [ ]:
adata = adata[adata.obs["multi_chain"] != "True", :].copy()

In [ ]:
sc.pl.umap(adata, color='leiden',use_raw=False,palette=sc.pl.palettes.default_102)

In [ ]:
plt.rcParams['figure.figsize']=(8,6)
sc.pl.umap(adata, color=["has_ir", "CD3E"])

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Compute CDR3 neighborhood graph and define clonotypes

`scirpy.pp.ir_neighbors()` computes a neighborhood graph based on CDR3 nucleotide (`nt`) or amino acid (`aa`) sequences, either based on sequence identity or similarity. Here, we define clonotypes based on `nt-sequenc`e identity.

In [ ]:
ir.pp.ir_neighbors(adata, receptor_arms="all",sequence='nt', dual_ir="primary_only")

In [ ]:
Define clonotypes by nucleotide sequence identity.

In [ ]:
ir.tl.define_clonotypes(adata)

Using `scirpy.tl.clonotype_network()` to compute the positions of cells in clonotype network and plot them colored by different parameters. Its recommended to set the min_size parameter to >=2, to prevent the singleton clonotypes from cluttering the network.

In [ ]:
ir.tl.clonotype_network(adata, min_size=2)
ir.pl.clonotype_network(adata, color="clonotype", legend_loc="none")

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Re-compute CDR3 neighborhood graph and define clonotype clusters

Now we re-compute the neighborhood graph based on amino-acid sequence similarity and define clonotype clusters. We are setting `metric="alignment"` and specifying a cutoff parameter. The distance is based on the BLOSUM62 matrix. All cells with a distance between their CDR3 sequences lower than `cutoff` will be connected in the network.

In [ ]:
ir.pp.ir_neighbors(
    adata,
    metric="alignment",
    sequence="aa",
    cutoff=15,
    receptor_arms="all",
    dual_ir="all",
)
ir.tl.define_clonotype_clusters(
    adata, partitions="connected", sequence="aa", metric="alignment", within_group=None
)

In [ ]:
ir.tl.clonotype_network(adata, min_size=4, sequence="aa", metric="alignment")

In [ ]:
ir.pl.clonotype_network(
    adata,
    color="ct_cluster_aa_alignment",
    legend_fontoutline=3,
    size=80,
    panel_size=(10, 9),
    legend_loc="on data"
)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Including the V-gene in clonotype definition

Using the paramter use_v_gene in `define_clonotypes()` , we can enforce clonotypes (or clonotype clusters) to have the same V-gene, and, therefore, the same CDR1 and 2 regions. Let’s look for clonotype clusters with different V genes:

In [ ]:
ir.tl.define_clonotype_clusters(
    adata,
    sequence="aa",
    metric="alignment",
    same_v_gene="primary_only",
    key_added="ct_cluster_aa_alignment_same_v",
)

In [ ]:
# find clonotypes with more than one `clonotype_same_v`
ct_different_v = adata.obs.groupby("ct_cluster_aa_alignment").apply(
    lambda x: x["ct_cluster_aa_alignment_same_v"].unique().size > 1
)
ct_different_v = ct_different_v[ct_different_v].index.values
ct_different_v

In [ ]:
# Display the first 2 clonotypes with different v genes
adata.obs.loc[
    adata.obs["ct_cluster_aa_alignment"].isin(ct_different_v[:2]),
    [
        "ct_cluster_aa_alignment",
        "ct_cluster_aa_alignment_same_v",
        "IR_VJ_1_v_gene",
        "IR_VDJ_1_v_gene",
    ],
].sort_values("ct_cluster_aa_alignment").drop_duplicates().reset_index(drop=True)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Clonotype analysis

#### Clonal expansion

Visualize the number of expanded clonotypes (i.e. clonotypes consisting of more than one cell) by cell-type. The first option is to add a column with the `scirpy.tl.clonal_expansion()` to adata.obs and overlay it on the UMAP plot.

Clonal_expansion refers to expansion categories, i.e singleton clonotypes, clonotypes with 2 cells and more than 2 cells. The clonotype_size refers to the absolute number of cells in a clonotype.

In [ ]:
ir.tl.clonal_expansion(adata)

In [ ]:
sc.pl.umap(adata, color=["clonal_expansion", "clonotype_size"])

The second option is to show the number of cells belonging to an expanded clonotype per category in a stacked bar plot, using the `scirpy.pl.clonal_expansion()` plotting function.

In [ ]:
ir.pl.clonal_expansion(adata, groupby="leiden", clip_at=4, normalize=False,figsize=(8,6))

The same plot, normalized to cluster size. Clonal expansion is a sign of positive selection for a certain, reactive T-cell clone. 

In [ ]:
ir.pl.clonal_expansion(adata, groupby="leiden", clip_at=4, normalize=True,figsize=(8,6))

Checking alpha diversity per cluster using `scirpy.pl.alpha_diversity()` of clonotypes.

In [ ]:
ir.pl.alpha_diversity(adata, groupby="leiden",figsize=(8,6))

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

#### Clonotype abundance

The function `scirpy.pl.group_abundance()` allows us to create bar charts for arbitrary categorial from obs. Here, we use it to show the distribution of the ten largest clonotypes across the cell-type clusters.

In [ ]:
ir.pl.group_abundance(adata, groupby="clonotype", target_col="leiden", max_cols=20,figsize=(8,6))

It might be beneficial to normalize the counts to the number of cells per sample to mitigate biases due to different sample sizes:

In [ ]:
ir.pl.group_abundance(
    adata, groupby="clonotype", target_col="leiden", max_cols=20, figsize=(8,6)
)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Gene usage

`scirpy.tl.group_abundance()` can also give us some information on VDJ usage. We can choose any of the `{TRA,TRB}_{1,2}_{v,d,j,c}_gene` columns to make a stacked bar plot. We use max_col to limit the plot to the 10 most abundant V-genes.

Plotting `gene_abundance` for all the non-zero `{TRA,TRB}_{1,2}_{v,d,j,c}_gene`

In [ ]:
ir_v_genes_list = [
  i for i in list(adata.obs.columns)
    if i.startswith('IR_V') and i.endswith('_gene')]
filtered_ir_v_genes_list = list()
for i in ir_v_genes_list:
  if adata.obs[i].shape[0] > adata.obs[i].isna().sum():
    filtered_ir_v_genes_list.append(i)

In [ ]:
for i in range(len(filtered_ir_v_genes_list)):
  ir.pl.group_abundance(
      adata,
      groupby=filtered_ir_v_genes_list[i],
      target_col="leiden",
      normalize=True,
      max_cols=20,
      fig_kws={'figsize':(10,8)})

The exact combinations of VDJ genes can be visualized as a Sankey-plot using `scirpy.pl.vdj_usage()`.

In [ ]:
ir.pl.vdj_usage(adata, full_combination=False, max_segments=None, max_ribbons=20,fig_kws={'figsize':(10,8)})

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

### Spectratype plots

`spectratype()` plots give us information about the length distribution of CDR3 regions.

In [ ]:
ir.pl.spectratype(adata, color="leiden", viztype="bar", fig_kws={"dpi": 120})

The same chart visualized as “ridge”-plot:

In [ ]:
ir.pl.spectratype(
    adata,
    color="leiden",
    viztype="curve",
    curve_layout="shifted",
    fig_kws={"dpi": 120},
    kde_kws={"kde_norm": False},
)

<div align="right"><a href="#Table-of-contents">Go to TOC</a></div>

## References
* [Analysis of 3k T cells from cancer](https://icbi-lab.github.io/scirpy/tutorials/tutorial_3k_tcr.html)
* [Scanpy - Preprocessing and clustering 3k PBMCs](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html)
* [single-cell-tutorial](https://github.com/theislab/single-cell-tutorial)

## Acknowledgement
The Imperial BRC Genomics Facility is supported by NIHR funding to the Imperial Biomedical Research Centre.